In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertTokenizer, TFBertModel, AutoTokenizer,TFAutoModel
import tensorflow as tf
from datasets import load_dataset
#import plotlib as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings

warnings.filterwarnings('ignore') # ignore Jupiter warnings

In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [3]:
model_Bert = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_Bert)
model = TFAutoModel.from_pretrained(model_Bert)

In [4]:
train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [5]:
train.head()

In [6]:
test.head()

In [7]:
def load_mnli(use_validation=True):
    result=[]
    dataset=load_dataset('multi_nli')
    print(dataset)
    for record in dataset['train']:
        c1, c2, c3 = record['premise'],record['hypothesis'], record['label']
        if c1 and c2 and c3 in {0, 1, 2}:
            result.append((c1, c2, c3, 'en'))
    result=pd.DataFrame(result, columns=['premise', 'hypothesis', 'label', 'lang_abv'])
    return result

In [8]:
mnli=load_mnli()
mnli

In [9]:
train = train[['premise', 'hypothesis', 'lang_abv', 'label']]
train

In [10]:
mnli=mnli[['premise', 'hypothesis', 'lang_abv', 'label']]
mnli

In [11]:
train=pd.concat([train, mnli.loc[:20000]], axis=0)

In [12]:
train.shape

In [13]:
SEQ_LEN = 50 #max(train.astype('str').applymap(lambda x: len(x)).max())

def bert_encode(df, tokenizer):    
    batch_premises = df['premise'].tolist()
    batch_hypothesis = df['hypothesis'].tolist()

    tokens = tokenizer(batch_premises, batch_hypothesis, max_length = SEQ_LEN,
                   truncation=True, padding='max_length',
                   add_special_tokens=True, return_attention_mask=True,
                   # return_token_type_ids=True, only for BERT
                   return_tensors='tf')
    inputs = {
          'input_ids': tokens['input_ids'], 
          'attention_mask': tokens['attention_mask'],
          } # 'token_type_ids': tokens['token_type_ids']    only for BERT
    return inputs

In [14]:
train_input = bert_encode(train, tokenizer)

In [15]:
from tensorflow.keras import regularizers

def build_model():

    encoder = TFAutoModel.from_pretrained(model_Bert)
    input_ids = tf.keras.Input(shape=(SEQ_LEN,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(SEQ_LEN,), dtype=tf.int32, name="attention_mask")
    #token_type_ids = tf.keras.Input(shape=(SEQ_LEN,), dtype=tf.int32,  name="token_type_ids") 
        
    embedding = encoder([input_ids, attention_mask])[0] #, token_type_ids  
    inputs=[input_ids, attention_mask] # , token_type_ids  
    #x = tf.keras.layers.Dense(256, activation=tf.nn.relu, kernel_regularizer=regularizers.l1(l1=1e-2))(embedding[:,0,:])
    output = tf.keras.layers.Dense(3,  activation='softmax')(embedding[:,0,:])
      
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-6), loss='sparse_categorical_crossentropy', metrics=['accuracy'])   
    return model 


In [16]:
with strategy.scope(): # defines the compute distribution policy for building the model. or in other words: makes sure that the model is created on the TPU/GPU/CPU, depending on to what the Accelerator is set in the Notebook Settings
    model = build_model() # our model is being built
    model.summary()       # let's look at some of its properties

In [17]:
for key in train_input.keys():
    train_input[key] = train_input[key][:,:SEQ_LEN]

In [18]:
#early_stop = tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
# make sure that TPU in Accelerator under Notebook Settings is turned on so that model trains on the TPU. Otherwise this line will crash
#hist = History()train.label.values
#labels = tf.cast(train['label'], dtype=tf.int32)
history = model.fit(train_input, train['label'], epochs = 10, batch_size=128, 
                    validation_split = 0.1) #,callbacks=[hist]) verbose = 1,         
print(history.history.keys())        

In [19]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [20]:
plt.clf() #Очистить рисунок
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [21]:
test_input = bert_encode(test, tokenizer) 

In [22]:
 #same for the test set we need to put it in the same size of the model
for key in test_input.keys():
    test_input[key] = test_input[key][:,:SEQ_LEN]

In [23]:
 predictions = [np.argmax(i) for i in model.predict(test_input)]

In [24]:
submission = test.id.copy().to_frame()
submission['prediction'] = predictions

In [25]:
submission.to_csv("./submission.csv", index = False)


In [26]:
submission.head()